## Notebook for meta analysis of experiments to determine exclusions

In [1]:
%run ./rob_setup_notebook.ipynb

from ia_funcs import *


Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']
QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Plots 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders


### Define list of pickled objects to meta analyse

In [2]:
pkl_list = [
            '2019-06-11_J048.pkl',
            '2019-06-13_J052.pkl',
            '2019-06-25_RL048.pkl',
            '2019-07-15_RL052.pkl',
            '2019-08-14_J058.pkl',
            '2019-08-14_J059.pkl',
            '2019-08-15_RL055.pkl',
            '2019-11-14_J061.pkl',
            '2019-11-14_J063.pkl',
            '2019-11-15_J064.pkl',
            '2019-11-18_J065.pkl',
            '2020-02-27_RL075.pkl',
            '2020-03-03_RL074.pkl',
            '2020-03-05_RL070.pkl',
            '2020-03-05_RL072.pkl',
            '2020-03-12_RL070.pkl',
            '2020-03-16_RL074.pkl',
            '2020-03-19_RL075.pkl',
#             '2020-03-20_RL080.pkl'
           ]

for i,pkl in enumerate(pkl_list):
    pkl_list[i] = os.path.join(qnap_path, 'pkl_files', pkl)

### Make STA movies

In [ ]:
staMovie(stam_save_path, pkl_list)

### Plot fluorescence across all concatenated t-series frames

In [ ]:
cellFluTime(pkl_list)
plt.savefig(os.path.join(fig_save_path, 'cell_fluorescence_time.svg'));

### Make downsampled stack of first 1000 and last 1000 frames of each experiment for all sessions

In [ ]:
for pkl in pkl_list:

        with open(pkl_path, 'rb') as f:
            exp_obj = pickle.load(f)
            
        obj_list = [exp_obj.photostim_r, exp_obj.photostim_s]

        if exp_obj.spont.n_frames > 0:
            obj_list.append(exp_obj.spont)

        if exp_obj.whisker_stim.n_frames > 0:
            obj_list.append(exp_obj.whisker_stim)
        
        for sub_obj in obj_list:
            
            tiff_path = sub_obj.tiff_path
            
            file_list = os.listdir(tiff_path)
            for file in file_list:
                if '.tif' in file:
                    tiff_file = os.path.join(tiff_path, file)
  
            total_frames = range(0,sub_obj.n_frames) #get the range of frames for this experiment
            start_frames = total_frames[30:1000] # get 970 frames after start
            end_frames = total_frames[-1000:-30] # get 970 frames before end
            
            stack_start = tf.imread(tiff_file, key=start_frames)
            stack_end = tf.imread(tiff_file, key=end_frames)
            
            mean_start = np.mean(stack_start, axis=0)
            mean_end = np.mean(stack_end, axis=0)
            
            save_path = os.path.join(qnap_path, 'Analysis', 'Drift_check', 'Raw', tiff_path.split('/')[-1])
            
            tf.imsave(save_path + '_mean_start.tif', mean_start.astype('int16'))
            tf.imsave(save_path + '_mean_end.tif', mean_end.astype('int16'))

### Plot frame fluorescence over time for each experiment in all sessions

In [ ]:
data_folder = os.path.join(qnap_path, 'Analysis', 'Drift_check', 'Stacks')
frameFluTime(pkl_list, data_folder, legend=False)
plt.savefig(os.path.join(fig_save_path, 'frame_fluorescence_time.svg'));